In [1]:
import sqlite3

conn = sqlite3.connect('lahman2015.sqlite')

Original dataquest query, required by the answer scorer, in mission:
https://www.dataquest.io/m/169/i%2Fo-bound-programs/12/returning-values-from-threads

I'm suggesting that the dataquest version is wrong as it is not summing the player's stats over time.

In [4]:
cur = conn.cursor()
query = """
    SELECT 
        (CAST(A AS FLOAT) + PO) / G as RF,  
        playerID
    FROM Fielding
    GROUP BY Fielding.playerID
    HAVING G > 100
    ORDER BY RF desc
    LIMIT 20;
    """
cur.execute(query)
res = cur.fetchall()
for r in res:
    print(r)

(11.869158878504672, 'chaseha01')
(11.868217054263566, 'phillbi01')
(11.784313725490197, 'daubeja01')
(11.65891472868217, 'stovage01')
(11.366906474820144, 'lesliro01')
(11.088709677419354, 'sweenbi03')
(10.946564885496183, 'werdepe01')
(10.903703703703703, 'jonesto01')
(10.898148148148149, 'ganzejo01')
(10.646017699115044, 'sharpbu01')
(10.625850340136054, 'siebedi01')
(10.556451612903226, 'unglabo01')
(10.470149253731343, 'dillopo01')
(10.459854014598541, 'lehanmi01')
(10.3, 'bossha01')
(10.25, 'nealoji01')
(10.226086956521739, 'gandich01')
(10.204724409448819, 'farrasi01')
(10.133858267716535, 'tucketo01')
(10.097902097902098, 'sheelea01')



Version with aggregate functions explicitly added to the query. Note that the top scorer on the dataquest version of the output, chaseha01, is missing entirely.


In [9]:
query = """
    SELECT 
        (sum(CAST(A AS FLOAT)) + sum(PO)) / sum(G) as RF,  
        playerID
    FROM Fielding
    GROUP BY Fielding.playerID
    HAVING sum(G) > 100
    ORDER BY RF desc
    LIMIT 20;
    """
cur.execute(query)
res = cur.fetchall()
for r in res:
    print(r)

(12.254838709677419, 'dehlmhe01')
(11.265151515151516, 'donahji02')
(11.090909090909092, 'lesliro01')
(10.987770460959549, 'jonesto01')
(10.973214285714286, 'sullich01')
(10.957119296316657, 'pippwa01')
(10.9079754601227, 'abstebi01')
(10.889908256880734, 'cogswed01')
(10.885741718674987, 'koneted01')
(10.864951768488746, 'dillopo01')
(10.739530988274707, 'holkewa01')
(10.734002509410288, 'terrybi01')
(10.720108695652174, 'ganzejo01')
(10.69767441860465, 'nealoji01')
(10.68612334801762, 'todtph01')
(10.658536585365853, 'newnapa01')
(10.65376782077393, 'rossmcl01')
(10.64, 'startjo01')
(10.631455399061032, 'grimera01')
(10.616129032258064, 'swaciha01')



And finally a simple select query where the calculated stat is produced for each row in the fielding table (i.e. for each year/stint/team combo for the selected fielder). Just looking at chaseha01 for sake of example here.


In [12]:
query = "select playerID, yearID, A, PO, G, (CAST(A AS FLOAT) + PO) / G as RF from fielding where playerID = 'chaseha01'"
cur.execute(query)
res = cur.fetchall()
for r in res:
    print(r)

('chaseha01', 1905, 61, 1174, 124, 9.959677419354838)
('chaseha01', 1905, 1, 1, 1, 2.0)
('chaseha01', 1905, 0, 3, 2, 1.5)
('chaseha01', 1906, 89, 1507, 150, 10.64)
('chaseha01', 1906, 3, 2, 1, 5.0)
('chaseha01', 1907, 77, 1144, 121, 10.090909090909092)
('chaseha01', 1907, 2, 8, 4, 2.5)
('chaseha01', 1908, 54, 1020, 98, 10.959183673469388)
('chaseha01', 1908, 7, 2, 3, 3.0)
('chaseha01', 1908, 4, 2, 1, 6.0)
('chaseha01', 1908, 0, 2, 3, 0.6666666666666666)
('chaseha01', 1908, 0, 0, 1, 0.0)
('chaseha01', 1909, 71, 1202, 118, 10.788135593220339)
('chaseha01', 1909, 0, 0, 1, 0.0)
('chaseha01', 1910, 65, 1373, 130, 11.061538461538461)
('chaseha01', 1911, 82, 1257, 124, 10.798387096774194)
('chaseha01', 1911, 3, 3, 2, 3.0)
('chaseha01', 1911, 0, 11, 7, 1.5714285714285714)
('chaseha01', 1912, 79, 1162, 122, 10.172131147540984)
('chaseha01', 1912, 10, 11, 7, 3.0)
('chaseha01', 1913, 16, 312, 29, 11.310344827586206)
('chaseha01', 1913, 3, 10, 5, 2.6)
('chaseha01', 1913, 0, 9, 5, 1.8)
('chaseha01'

Notice that the last number in this final query is the same as the value returned for this player by the DataQuest query: 11.869158878504672. This shows that the dataquest query is actually just getting the player's best year, not the player's overall score.